# Estimador de Popesso

In [93]:
import numpy as np
import pandas as pd 
from astropy.stats import biweight_scale
from Estimadores import PosicionesRelativas

#Constante de gravitación (SI)
G = 6.6743e-11
#Masa solar (kg)
MS = 1.9891e30

In [94]:
#Dirección de los datos
dirr = 'Datos\A2255_dccoutcut.csv'

#Distancia al cúmulo (m) (los he estado sacando de NED)
d = 1.093657e+25


Popesso utiliza la dispersión de velocidades por medio del estimador biwight (Beers et al. 1990, ecuación 9)

Este estimador está hecho en la librería Astropy.

Además, considera las correcciones de The & White con el término superficial. 

In [95]:
# Desviación estándar de las velocidades por medio del estimador biweight
ARrel, DECrel, N, Vrel = PosicionesRelativas(dirr)

#Desviación estándar
sigma = biweight_scale(Vrel)
sigma2 = sigma**2
#sigma = 7160000
print(sigma)
print(sigma2)

978577.076407328
957613094469.9135


La masa virial es corregida por el término de presión superficial de The & White (1986) adoptando un perfil de Navarro et al. (1997, 1997).

Tenemos una especie de generalización del teorema del Virial con constricciones incluidas (The & White (1986)):

$<v^2> -  \dfrac{G M_{t, o}}{r_o} I = Término\; \; Superficial   $

donde se adopta $Término\; \; Superficial = 0.21 <v^2>$ e $I$ es un coeficiente proveniente de una integral, la cual da como resultado $I = 1.28$

$<v^2> -  \dfrac{G M_{t, o}}{r_o} I =  0.21 <v^2> \implies M = \dfrac{(1 -0.21)<v^2>}{G I} r_0$

De acuerdo con Girarldi et al. 1998, la masa virial está dada por:

$M_{virial} = \dfrac{3\pi}{2} \dfrac{\sigma^2 R_{pv}}{G

donde $sigma^2$ será calculada con el estimador biweight y $R_{pv}$ es el radio virial proyectado, dado por:

$R_{pv} = \displaystyle \dfrac{N(N-1) }{\sum_{i<j} 1/R_{ij}}$

In [96]:
#Diferencia angular de galaxias
#Diferencia angular entre galaxias
theta = []

for i in range(N):
    for j in range(i+1, N):
        ARdif = ARrel[i] - ARrel[j]   
        DECdif = DECrel[i] - DECrel[j]
        theta.append( 1/np.sqrt( ARdif**2 + DECdif**2) )  
        #print("(",i,",", j, ")", end='\t')
       #print(i==j, end='\t')
    
#Sumamos el resultado
SumTheta = sum(theta)

In [97]:
#Estimación virial con desviación de biweight
Mv = (1-0.21)*(3*np.pi*d*N*(N-1)*sigma2)/(2*G*SumTheta*0.24)
print("{:.2e}".format(Mv/MS))

1.07e+16


In [98]:
#Sacamos el r200
#radio de apertura (radio de Abell)
rap = 6.6342e+22
#Densidad crítica (del carroll) en teoría debería ser función del redshioft
rhoc = 9.47e-27 
#Densidad virial
rhovir = 3*Mv/(4*np.pi*(rap**3))

r200 = rap*( rhovir/(200*rhoc))**(1/2.4)
r200/3.086e22

5.422163294220419

In [99]:
H0 = 2.30e-18
M200 = (100*(r200**3)*(H0**2) )/G
print("{:.2e}".format(M200/MS))

1.87e+16


In [101]:
1.87/5.4

0.34629629629629627